# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
IEX_CLOUD_API_TOKEN

'Tpk_fc9b0811428e4ff3b7e0c4685b6448c6'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2123066476084,
 'week52high': 145.66,
 'week52low': 54.98,
 'week52highSplitAdjustOnly': 152.24,
 'week52lowSplitAdjustOnly': 54.18,
 'week52change': 0.8110022777911551,
 'sharesOutstanding': 17328538481,
 'float': 0,
 'avg10Volume': 139932480,
 'avg30Volume': 100037450,
 'day200MovingAvg': 126.04,
 'day50MovingAvg': 135.78,
 'employees': 149705,
 'ttmEPS': 3.79,
 'ttmDividendRate': 0.8418029465791004,
 'dividendYield': 0.007036338787271969,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-24',
 'nextEarningsDate': '2021-01-24',
 'peRatio': 32.612967535404955,
 'beta': 1.13672936204968,
 'maxChangePercent': 46.4790828042336,
 'year5ChangePercent': 4.549738428926766,
 'year2ChangePercent': 1.875731554042843,
 'year1ChangePercent': 0.8175671446440427,
 'ytdChangePercent': -0.08731031993958045,
 'month6ChangePercent': -0.05878311962474161,
 'month3ChangePercent': 0.020085808118970785,
 'month1ChangePercent': -0.08078210026441623,
 'day30Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.8175671446440427

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

Ticker   Price One-Year Price Return Number of Shares to Buy
0        A  129.10              0.617952                     N/A
1      AAL   21.74              0.100404                     N/A
2      AAP  164.94              0.217013                     N/A
3     AAPL  129.08               0.81578                     N/A
4     ABBV  109.93              0.336891                     N/A
..     ...     ...                   ...                     ...
500    YUM  107.09              0.187428                     N/A
501    ZBH  164.80              0.213114                     N/A
502   ZBRA  515.84              1.379085                     N/A
503   ZION   55.41              0.394375                     N/A
504    ZTS  161.49              0.170235                     N/A

[505 rows x 4 columns]

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

Ticker    Price One-Year Price Return Number of Shares to Buy
0     FCX    36.10              2.426275                     N/A
1    CARR    37.50              2.142797                     N/A
2    VIAC    67.45              1.752779                     N/A
3    ALGN   576.32               1.63132                     N/A
4      LB    57.26              1.598107                     N/A
5    SIVB   553.42              1.498511                     N/A
6    PYPL   278.51              1.429478                     N/A
7    ZBRA   515.84              1.379085                     N/A
8    TWTR    79.29              1.332712                     N/A
9      DE   363.17              1.314822                     N/A
10    URI   314.91              1.275893                     N/A
11    PWR    89.80              1.230708                     N/A
12   CDNS   150.48              1.188517                     N/A
13   ABMD   329.34              1.168701                     N/A
14   KLAC   340.48              1.106801                     N/A
15  DISCA    59.51              1.081846                     N/A
16   NVDA   571.33              1.074003                     N/A
17   AMAT   126.24              1.072816                     N/A
18   IDXX   552.92              1.060077                     N/A
19   LRCX   605.48              0.999353                     N/A
20   NWSA    24.89              0.996205                     N/A
21    ALB   159.26              0.987544                     N/A
22   APTV   155.94              0.955028                     N/A
23    NWS    24.75              0.908989                     N/A
24    WST   294.42               0.90324                     N/A
25    CMG  1497.99               0.89349                     N/A
26    AMD    90.29              0.892483                     N/A
27    EMN   112.83              0.880255                     N/A
28    FDX   262.53              0.868205                     N/A
29    TPR    42.00              0.860195                     N/A
30   TSCO   166.19              0.856465                     N/A
31   AVGO   491.05              0.839696                     N/A
32    TGT   193.63              0.835477                     N/A
33    UPS   169.07              0.816433                     N/A
34   AAPL   129.08               0.81578                     N/A
35   QCOM   140.91              0.814155                     N/A
36   IPGP   233.54              0.810138                     N/A
37   SWKS   188.07              0.803685                     N/A
38  DISCK    50.03              0.800106                     N/A
39    CAT   229.98              0.796812                     N/A
40   SNPS   257.69              0.791972                     N/A
41    MOS    31.54               0.77069                     N/A
42     MS    81.37              0.766907                     N/A
43     MU    98.57              0.765904                     N/A
44    GPS    26.66              0.756193                     N/A
45   QRVO   187.77              0.746254                     N/A
46   MKTX   597.20              0.742575                     N/A
47   MXIM    97.88              0.728017                     N/A
48    CMI   275.90              0.725944                     N/A
49   MCHP   161.80              0.725386                     N/A
50   TTWO   193.25              0.724614                     N/A

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:100000
100000


In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/home/actserv/Documents/upwork_data_science/env/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Ticker    Price One-Year Price Return Number of Shares to Buy
0     FCX    36.10              2.426275                      54
1    CARR    37.50              2.142797                      52
2    VIAC    67.45              1.752779                      29
3    ALGN   576.32               1.63132                       3
4      LB    57.26              1.598107                      34
5    SIVB   553.42              1.498511                       3
6    PYPL   278.51              1.429478                       7
7    ZBRA   515.84              1.379085                       3
8    TWTR    79.29              1.332712                      24
9      DE   363.17              1.314822                       5
10    URI   314.91              1.275893                       6
11    PWR    89.80              1.230708                      21
12   CDNS   150.48              1.188517                      13
13   ABMD   329.34              1.168701                       5
14   KLAC   340.48              1.106801                       5
15  DISCA    59.51              1.081846                      32
16   NVDA   571.33              1.074003                       3
17   AMAT   126.24              1.072816                      15
18   IDXX   552.92              1.060077                       3
19   LRCX   605.48              0.999353                       3
20   NWSA    24.89              0.996205                      78
21    ALB   159.26              0.987544                      12
22   APTV   155.94              0.955028                      12
23    NWS    24.75              0.908989                      79
24    WST   294.42               0.90324                       6
25    CMG  1497.99               0.89349                       1
26    AMD    90.29              0.892483                      21
27    EMN   112.83              0.880255                      17
28    FDX   262.53              0.868205                       7
29    TPR    42.00              0.860195                      46
30   TSCO   166.19              0.856465                      11
31   AVGO   491.05              0.839696                       3
32    TGT   193.63              0.835477                      10
33    UPS   169.07              0.816433                      11
34   AAPL   129.08               0.81578                      15
35   QCOM   140.91              0.814155                      13
36   IPGP   233.54              0.810138                       8
37   SWKS   188.07              0.803685                      10
38  DISCK    50.03              0.800106                      39
39    CAT   229.98              0.796812                       8
40   SNPS   257.69              0.791972                       7
41    MOS    31.54               0.77069                      62
42     MS    81.37              0.766907                      24
43     MU    98.57              0.765904                      19
44    GPS    26.66              0.756193                      73
45   QRVO   187.77              0.746254                      10
46   MKTX   597.20              0.742575                       3
47   MXIM    97.88              0.728017                      20
48    CMI   275.90              0.725944                       7
49   MCHP   161.80              0.725386                      12
50   TTWO   193.25              0.724614                      10

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [56]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
convert_none = lambda x : 0 if x is None else x

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    convert_none(data[symbol]['stats']['year1ChangePercent']),
                    'N/A',
                    convert_none(data[symbol]['stats']['month6ChangePercent']),
                    'N/A',
                    convert_none(data[symbol]['stats']['month3ChangePercent']),
                    'N/A',
                    convert_none(data[symbol]['stats']['month1ChangePercent']),
                    'N/A'
                ],
                index = hqm_columns
            ),
            ignore_index=True
        )        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile'],
      dtype='object')

In [57]:
hqm_dataframe

Ticker   Price Number of Shares to Buy  One-Year Price Return  \
0        A  129.22                     N/A               0.626366   
1      AAL   21.58                     N/A               0.103552   
2      AAP  166.62                     N/A               0.219691   
3     AAPL  131.93                     N/A               0.793077   
4     ABBV  109.21                     N/A               0.337721   
..     ...     ...                     ...                    ...   
500    YUM  107.07                     N/A               0.186890   
501    ZBH  169.80                     N/A               0.214981   
502   ZBRA  523.89                     N/A               1.427725   
503   ZION   55.12                     N/A               0.398022   
504    ZTS  163.46                     N/A               0.173390   

    One-Year Return Percentile  Six-Month Price Return  \
0                          N/A                0.223391   
1                          N/A                0.605047   
2                          N/A                0.029416   
3                          N/A               -0.058119   
4                          N/A                0.159697   
..                         ...                     ...   
500                        N/A                0.092567   
501                        N/A                0.166121   
502                        N/A                0.764281   
503                        N/A                0.705818   
504                        N/A               -0.029471   

    Six-Month Return Percentile  Three-Month Price Return  \
0                           N/A                  0.046257   
1                           N/A                  0.486700   
2                           N/A                  0.088266   
3                           N/A                  0.020822   
4                           N/A                  0.043832   
..                          ...                       ...   
500                         N/A                 -0.016795   
501                         N/A                  0.095519   
502                         N/A                  0.331007   
503                         N/A                  0.394750   
504                         N/A                 -0.032958   

    Three-Month Return Percentile  One-Month Price Return  \
0                             N/A                0.016172   
1                             N/A                0.225589   
2                             N/A                0.076656   
3                             N/A               -0.081234   
4                             N/A                0.053105   
..                            ...                     ...   
500                           N/A                0.025006   
501                           N/A                0.061925   
502                           N/A                0.293199   
503                           N/A                0.216907   
504                           N/A                0.006732   

    One-Month Return Percentile  
0                           N/A  
1                           N/A  
2                           N/A  
3                           N/A  
4                           N/A  
..                          ...  
500                         N/A  
501                         N/A  
502                         N/A  
503                         N/A  
504                         N/A  

[505 rows x 11 columns]

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [58]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
            hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# # Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

Ticker   Price Number of Shares to Buy  One-Year Price Return  \
0        A  129.22                     N/A               0.626366   
1      AAL   21.58                     N/A               0.103552   
2      AAP  166.62                     N/A               0.219691   
3     AAPL  131.93                     N/A               0.793077   
4     ABBV  109.21                     N/A               0.337721   
..     ...     ...                     ...                    ...   
500    YUM  107.07                     N/A               0.186890   
501    ZBH  169.80                     N/A               0.214981   
502   ZBRA  523.89                     N/A               1.427725   
503   ZION   55.12                     N/A               0.398022   
504    ZTS  163.46                     N/A               0.173390   

    One-Year Return Percentile  Six-Month Price Return  \
0                     0.857426                0.223391   
1                     0.293069                0.605047   
2                     0.455446                0.029416   
3                     0.928713               -0.058119   
4                     0.609901                0.159697   
..                         ...                     ...   
500                   0.411881                0.092567   
501                   0.449505                0.166121   
502                   0.986139                0.764281   
503                   0.679208                0.705818   
504                   0.388119               -0.029471   

    Six-Month Return Percentile  Three-Month Price Return  \
0                      0.582178                  0.046257   
1                      0.879208                  0.486700   
2                      0.283168                  0.088266   
3                      0.136634                  0.020822   
4                      0.493069                  0.043832   
..                          ...                       ...   
500                     0.39604                 -0.016795   
501                     0.50297                  0.095519   
502                    0.940594                  0.331007   
503                    0.914851                  0.394750   
504                    0.180198                 -0.032958   

    Three-Month Return Percentile  One-Month Price Return  \
0                        0.459406                0.016172   
1                        0.972277                0.225589   
2                        0.538614                0.076656   
3                        0.384158               -0.081234   
4                        0.447525                0.053105   
..                            ...                     ...   
500                      0.275248                0.025006   
501                      0.556436                0.061925   
502                      0.891089                0.293199   
503                      0.936634                0.216907   
504                      0.241584                0.006732   

    One-Month Return Percentile  
0                       0.39802  
1                      0.906931  
2                       0.60396  
3                      0.059406  
4                      0.526733  
..                          ...  
500                    0.423762  
501                    0.554455  
502                    0.954455  
503                    0.887129  
504                    0.362376  

[505 rows x 11 columns]

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [60]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [61]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [62]:
portfolio_input()

Enter the value of your portfolio:1000000


In [63]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/home/actserv/Documents/upwork_data_science/env/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Ticker    Price Number of Shares to Buy  One-Year Price Return  \
0       A   129.22                     151               0.626366   
1     AAL    21.58                     908               0.103552   
2     AAP   166.62                     117               0.219691   
3    AAPL   131.93                     148               0.793077   
4    ABBV   109.21                     179               0.337721   
5     ABC   106.14                     184               0.222375   
6    ABMD   336.26                      58               1.178738   
7     ABT   124.56                     157               0.598829   
8     ACN   262.35                      74               0.415081   
9    ADBE   487.71                      40               0.336628   
10    ADI   159.47                     122               0.479387   
11    ADM    57.40                     341               0.555144   
12    ADP   177.50                     110               0.154690   
13   ADSK   297.02                      66               0.456424   
14    AEE    73.88                     265              -0.094979   
15    AEP    76.80                     255              -0.137417   
16    AES    28.00                     700               0.646317   
17    AFL    50.92                     385               0.144417   
18    AIG    46.13                     425               0.089229   
19    AIV     4.86                    4034              -0.226237   
20    AIZ   128.20                     152               0.040826   
21    AJG   123.20                     159               0.250300   
22   AKAM    98.96                     198               0.095055   
23    ALB   156.89                     124               0.977116   
24   ALGN   578.83                      33               1.663509   
25    ALK    64.83                     302               0.299541   
26    ALL   114.03                     171               0.030673   
27   ALLE   116.70                     168              -0.042730   
28   ALXN   159.41                     123               0.640673   
29   AMAT   126.17                     155               1.090345   
30   AMCR    11.50                    1705               0.244939   
31    AMD    90.41                     216               0.867941   
32    AME   123.27                     159               0.397646   
33   AMGN   238.32                      82               0.160962   
34    AMP   234.48                      83               0.631677   
35    AMT   221.83                      88              -0.029831   
36   AMZN  3232.15                       6               0.667667   
37   ANET   291.86                      67               0.455317   
38   ANSS   356.72                      54               0.412342   
39   ANTM   312.30                      62               0.198541   
40    AON   235.23                      83               0.469832   
41    AOS    63.19                     310               0.537498   
42    APA    19.93                     983              -0.199304   
43    APD   261.33                      75               0.197046   
44    APH   130.86                     149               0.388485   
45   APTV   162.16                     120               0.918379   
46    ARE   163.82                     119               0.083764   
47    ATO    88.07                     222              -0.170666   
48   ATVI   101.81                     192               0.686149   
49    AVB   178.34                     109              -0.088102   
50   AVGO   505.85                     N/A               0.831543   

   One-Year Return Percentile  Six-Month Price Return  \
0                    0.857426                0.223391   
1                    0.293069                0.605047   
2                    0.455446                0.029416   
3                    0.928713               -0.058119   
4                    0.609901                0.159697   
5                    0.463366                0.052800   
6              

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [64]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [65]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [66]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [67]:
writer.save()